In [0]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#change directory
cd /content/drive/My Drive

/content/drive/My Drive


In [0]:
import re
from textblob import TextBlob
import os
import numpy as np
import requests
import time

contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "I would",
"i'd've": "I would have",
"i'll": "I shall",
"i'll've": "I shall have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why has",
"why've": "why have",
"will've": "will have",
"won't": "would not",
"won't've": "would not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
"uiu": "united international university"
}

msg_dlt = '\"message\"'
pattern = re.compile(r"(.)\1{2,}", re.DOTALL)   #if more then 2 time an alphabet repeat then reduce them. for example heeeeee will be hee

def clean():
    filenames = []
    all_str = ""
    # Keep the all files of directory
    for f in os.listdir("fbData"):
        if f.endswith(".txt"):
            filenames.append(f)
    print(filenames)
    print(len(filenames))
    # after append all text file in filenames list

    numberOfPosts = []
    # Read the files line by line
    for i in range(0, len(filenames)):
        status = open("fbData" + '/' + filenames[i], 'r', encoding="utf-8", errors='replace')
        all_str = ''
        fbData = []
        engLangData = []
        otherLangData = []
        for line in status.readlines():
            match = re.search(msg_dlt, line)
            if match:
                all_str = line
                all_str = re.sub('\"message\":', '', all_str)           # remove all "message\:"
                all_str = all_str.lower()                               # convert text to lower-case
                all_str = re.sub(' [ ]+', ' ', all_str)                 # remove unnecessary space
                all_str = all_str.replace('\\n', ' ')                   # remove all \n
                all_str = re.sub('\?', ' ', all_str)       # remove all ? without last position of a line
                all_str = re.sub('\" *\",', '', all_str)                # remove all empty string " "
                all_str = re.sub(r'http\S+', '', all_str)               # remove all Url" "
                all_str = re.sub(r'#[ ]*([^\s]+)', r'\1', all_str)          # remove the # in #hashtag
                all_str = all_str.replace('\"',' ')                      # remove all "
                all_str = all_str.replace(',', ' ')                     # remove all ,
                all_str = re.sub('\.[\.]+', ' ', all_str)               # remove ...
                all_str = re.sub('\\\\', '', all_str)                   # remove backslash
                all_str = re.sub('�','\'', all_str)
                all_str = re.sub(' i ',' I ', all_str)


                #replace short word into full word E.g.. i m shazzad will be i am shazzad
                all_str = re.sub(r' r ', ' are ', all_str)
                all_str = re.sub(r' m ', ' am ', all_str)
                all_str = re.sub(r' u ', ' you ', all_str)
                all_str = re.sub(r' b ', ' be ', all_str)
                all_str = re.sub(r' n8 ', ' night ', all_str)
                all_str = re.sub(r' gn8 ', ' good night ', all_str)
                all_str = re.sub(r' r8 ', ' right ', all_str)
                all_str = re.sub(r' hv ', ' have ', all_str)
                all_str = re.sub(r' bt ', ' but ', all_str)
                all_str = re.sub(r' ur ', ' your ', all_str)
                all_str = re.sub(r' n ', ' and ', all_str)
                all_str = re.sub(r' bro ', ' brother ', all_str)
                all_str = re.sub(r' tha ', ' the ', all_str)
                all_str = re.sub(r' it(z)+ ', ' it\'s ', all_str)
                all_str = re.sub(r' ai ', ' artificial intelligent ', all_str)
                all_str = re.sub(r' uni ', ' university ', all_str)
                all_str = pattern.sub(r"\1\1", all_str)                #remove more then 2 charecter in a row.. like amiiii will be amii
                fbData.append(all_str)
        for j in range(0,len(fbData)):
            fbData[j] = fbData[j].strip()                               # remove all extra spaces
            for word in fbData[j].split():                              # conver i'm = i am
                if word.lower() in contractions:
                    fbData[j] = fbData[j].replace(word, contractions[word.lower()])
            if (len(fbData[j]) > 2):
                b = TextBlob(fbData[j])
                language = b.detect_language() 
            else:
                language = "error"
            if language=="en" or language=="error":
                engLangData.append(fbData[j])
            else:
                fbData[j] = fbData[j] + "---->"+ language
                otherLangData.append(fbData[j])
            print(str(j) +"file completed")
            time.sleep(.5)
        print("-----------------------------------"+str(j)+"---------------------------")
            

        engLangData = list(filter(None, engLangData))                             # remove empty string
        otherLangData = list(filter(None, otherLangData))                             # remove empty string
        #print(fbData)
        # writing to the content
        with open('cleanData' + '/' + filenames[i], 'w', encoding= "utf-8") as file:
            for item in engLangData:
                file.write("%s\n" % item)
        with open('cleanData' + '/' + 'otherLangData' + '/' + filenames[i], 'w', encoding= "utf-8") as file:
            for item in otherLangData:
                file.write("%s\n" % item)

        numberOfPosts.append(len(engLangData))



    numberOfPosts = np.asarray(numberOfPosts)
    AveragePostLength = np.average(numberOfPosts)
    print("Number of Post:", numberOfPosts)
    print("Average Number Of Post: ", AveragePostLength)

if __name__ == "__main__":
    clean()


['011151239.txt', '011151328.txt', '011151352.txt', '011152010.txt', '011152012 - Mehedi Hasan.txt', '011152011.txt', '011152022 - Md Ashraf Uddin Shahed.txt', '011152022.txt', '011152024.txt', '011152025.txt', '011152037.txt', '011152078.txt', '011152086.txt', '011152143.txt', '011152129.txt', '011152077.txt', '011153001.txt', '011152155.txt', '011152144.txt', '011153047.txt', '011153076.txt', '011153109 - Tashin Ahmed.txt', '011161018.txt', '011161126.txt', '011161073.txt', '011161040.txt', '011161123.txt', '011161027.txt', '011161121.txt', '011161128.txt', '011161125.txt', '011161132.txt', '011161136.txt', '011161161.txt', '011161174.txt', '011163039.txt', '011161195.txt', '011162009.txt', '011161197.txt', '011162123.txt', '011162139.txt', '011161274.txt', '011163071.txt', '011171141.txt', '011163075.txt', '011163077.txt', '011163092.txt', '011171031.txt', '011163096.txt', '011171145.txt', '011171054.txt', '011171032.txt', '011171106.txt', '011171151.txt', '011171153.txt', '01117122